# IDH1 Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "IDH1"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
IDH3G
ACO2
OGDH
ALDOC
LTA4H
CAT
GOT2
GLUD1
ACOX1
IDH3A
AGXT
ACOX2
ACO1
NOS2
IDH2
HBB
PGM1
MMP9
OGDHL
HMGCL
HAO1
IDH3B
ALDOA
IDH1
HSD17B4
HAO2
ELAVL1
SLC2A4
CDK2
ASF1B
PEX5
ANXA6
C12orf10
CBX3
CHMP4A
DCK
EFHD2
FDPS
GOT1
NME1
OXCT1
RANBP1
SBDS
THOP1


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

                                Comparison   P_Value
0  GLUD1_transcriptomics_ENSG00000148672.8  0.000022





# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  IDH1


Doing t-test comparisons

                                          Comparison       P_Value
0    RP11-168O22.1_transcriptomics_ENSG00000236744.2  1.676259e-22
1    RP11-264A11.1_transcriptomics_ENSG00000236958.1  4.112785e-20
2            FREM3_transcriptomics_ENSG00000183090.5  3.834287e-19
3            OR4N2_transcriptomics_ENSG00000176294.4  5.047973e-18
4           OR4K6P_transcriptomics_ENSG00000228304.1  8.123690e-18
..                                               ...           ...
454   RP11-46C24.7_transcriptomics_ENSG00000259877.2  7.915440e-07
455      LINC00693_transcriptomics_ENSG00000228214.5  7.943545e-07
456   RP4-635E18.7_transcriptomics_ENSG00000226849.1  8.025414e-07
457     AC000095.9_transcriptomics_ENSG00000270393.1  8.045376e-07
458   GS1-124K5.13_transcriptomics_ENSG00000279785.1  8.100096e-07

[459 rows x 2 columns]





# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,IDH1,GLUD1_transcriptomics_ENSG00000148672.8,True,2.152232e-05
0,Gbm,IDH1,RP11-168O22.1_transcriptomics_ENSG00000236744.2,False,1.676259e-22
1,Gbm,IDH1,RP11-264A11.1_transcriptomics_ENSG00000236958.1,False,4.112785e-20
2,Gbm,IDH1,FREM3_transcriptomics_ENSG00000183090.5,False,3.834287e-19
3,Gbm,IDH1,OR4N2_transcriptomics_ENSG00000176294.4,False,5.047973e-18
...,...,...,...,...,...
454,Gbm,IDH1,RP11-46C24.7_transcriptomics_ENSG00000259877.2,False,7.915440e-07
455,Gbm,IDH1,LINC00693_transcriptomics_ENSG00000228214.5,False,7.943545e-07
456,Gbm,IDH1,RP4-635E18.7_transcriptomics_ENSG00000226849.1,False,8.025414e-07
457,Gbm,IDH1,AC000095.9_transcriptomics_ENSG00000270393.1,False,8.045376e-07


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"